In [30]:
def Validation(n_fold,X):
    list_train_fold = []
    list_val_fold = []
    list_train = []
    Number = X.shape[0]//n_fold
    for i in range(X.shape[0]):
        list_train.append(i)
    
    for i in range(n_fold)[::-1]:
        list_val = []
        if i==n_fold-1:
            for j in range(Number*i,X.shape[0]):
                list_val.append(j)
                
            list_train_fold.append(np.setdiff1d(list_train,list_val))
            list_val_fold.append(list_val)
            
        
        if i != n_fold-1: 
            for j in range(Number*i,Number*(i+1)):
                list_val.append(j)
            list_train_fold.append(np.setdiff1d(list_train,list_val))
            list_val_fold.append(list_val)
        
    return list_train_fold, list_val_fold

In [31]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.insert(0, '..')
from lib import models_Path, graph, coarsening, utils
# modified
import tensorflow.compat.v1 as tf
import numpy as np
import time
import pandas as pd
%matplotlib inline
import random
import tensorflow as tf
tf.__version__


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'1.15.0'

In [54]:
def data_simulation(exp, ppi,p):
    dataset = {}
    all_nodes = exp.shape[1]
    dataset['exp'] = []
    dataset['L'] = []
    dataset['perm'] = []
    Pathway_Name = list(p)
    
        
    for i in range(len(Pathway_Name)):
        Node_Number = []
        Sub_Pathway = p[Pathway_Name[i]]['gene'].values()
        Numbering = list(Sub_Pathway)
        for a in range(len(Numbering)):
            for b in range(exp.shape[1]):
                if Numbering[a] == exp.columns[b]:
                    Node_Number.append(b)

        sample_ppi = ppi.iloc[Node_Number, Node_Number]
        sample_ppi = np.array(sample_ppi)
        print("PPI Shape :",len(Node_Number), sample_ppi.shape[0])
        A = coo_matrix(sample_ppi,dtype=np.float32)
        print("Number of Edge :",A.nnz//2, "//",i)
        
        graphs, perm = coarsening.coarsen(A, levels=4, self_connections=False)
        print(type(A))
        L = [graph.laplacian(A, normalized=True) for A in graphs]
        dataset['exp'].append(exp.iloc[:, Node_Number].values)
        dataset['L'].append(L)
        dataset['perm'].append(perm)
    return dataset

In [56]:
import pickle
from scipy.sparse import coo_matrix
ppi = pd.read_csv("Table/PPI.csv")
#ppi = pd.read_csv("Data/L1000/groupPPI.csv")
ppi.drop(['string'], axis='columns', inplace=True)
#ppi.drop(['Unnamed: 0'], axis='columns', inplace=True)

exp = pd.read_csv("Table/EXP.csv")
#exp = pd.read_csv("Data/L1000/groupEXP.csv")
exp.drop(['Unnamed: 0'], axis='columns', inplace=True)
print(exp.shape)
print(ppi.shape)
with open('Table/Pathway.pkl','rb')as f:
    p = pickle.load(f)



(734, 7545)
(7545, 7545)


In [58]:
dataset = data_simulation(exp, ppi,p)


PPI Shape : 35 35
Number of Edge : 479 // 0
Layer 0: M_0 = |V| = 48 nodes (13 added),|E| = 479 edges
Layer 1: M_1 = |V| = 24 nodes (6 added),|E| = 151 edges
Layer 2: M_2 = |V| = 12 nodes (3 added),|E| = 36 edges
Layer 3: M_3 = |V| = 6 nodes (1 added),|E| = 10 edges
Layer 4: M_4 = |V| = 3 nodes (0 added),|E| = 3 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 17 17
Number of Edge : 129 // 1
Layer 0: M_0 = |V| = 32 nodes (15 added),|E| = 129 edges
Layer 1: M_1 = |V| = 16 nodes (7 added),|E| = 36 edges
Layer 2: M_2 = |V| = 8 nodes (3 added),|E| = 10 edges
Layer 3: M_3 = |V| = 4 nodes (1 added),|E| = 3 edges
Layer 4: M_4 = |V| = 2 nodes (0 added),|E| = 1 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 26 26
Number of Edge : 248 // 2
Layer 0: M_0 = |V| = 32 nodes (6 added),|E| = 248 edges
Layer 1: M_1 = |V| = 16 nodes (3 added),|E| = 73 edges
Layer 2: M_2 = |V| = 8 nodes (1 added),|E| = 20 edges
Layer 3: M_3 = |V| = 4 nodes (0 added),|E| = 6 edges
Layer 4: M_4 = |V| = 2 node

PPI Shape : 40 40
Number of Edge : 482 // 24
Layer 0: M_0 = |V| = 48 nodes (8 added),|E| = 482 edges
Layer 1: M_1 = |V| = 24 nodes (3 added),|E| = 189 edges
Layer 2: M_2 = |V| = 12 nodes (1 added),|E| = 55 edges
Layer 3: M_3 = |V| = 6 nodes (0 added),|E| = 15 edges
Layer 4: M_4 = |V| = 3 nodes (0 added),|E| = 3 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 19 19
Number of Edge : 114 // 25
Layer 0: M_0 = |V| = 32 nodes (13 added),|E| = 114 edges
Layer 1: M_1 = |V| = 16 nodes (6 added),|E| = 36 edges
Layer 2: M_2 = |V| = 8 nodes (3 added),|E| = 10 edges
Layer 3: M_3 = |V| = 4 nodes (1 added),|E| = 3 edges
Layer 4: M_4 = |V| = 2 nodes (0 added),|E| = 1 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 24 24
Number of Edge : 186 // 26
Layer 0: M_0 = |V| = 32 nodes (8 added),|E| = 186 edges
Layer 1: M_1 = |V| = 16 nodes (3 added),|E| = 69 edges
Layer 2: M_2 = |V| = 8 nodes (1 added),|E| = 21 edges
Layer 3: M_3 = |V| = 4 nodes (0 added),|E| = 6 edges
Layer 4: M_4 = |V| = 2 no

PPI Shape : 57 57
Number of Edge : 589 // 47
Layer 0: M_0 = |V| = 64 nodes (7 added),|E| = 589 edges
Layer 1: M_1 = |V| = 32 nodes (2 added),|E| = 262 edges
Layer 2: M_2 = |V| = 16 nodes (1 added),|E| = 89 edges
Layer 3: M_3 = |V| = 8 nodes (0 added),|E| = 28 edges
Layer 4: M_4 = |V| = 4 nodes (0 added),|E| = 6 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 32 32
Number of Edge : 226 // 48
Layer 0: M_0 = |V| = 48 nodes (16 added),|E| = 226 edges
Layer 1: M_1 = |V| = 24 nodes (7 added),|E| = 92 edges
Layer 2: M_2 = |V| = 12 nodes (3 added),|E| = 29 edges
Layer 3: M_3 = |V| = 6 nodes (1 added),|E| = 9 edges
Layer 4: M_4 = |V| = 3 nodes (0 added),|E| = 3 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 31 31
Number of Edge : 209 // 49
Layer 0: M_0 = |V| = 32 nodes (1 added),|E| = 209 edges
Layer 1: M_1 = |V| = 16 nodes (0 added),|E| = 83 edges
Layer 2: M_2 = |V| = 8 nodes (0 added),|E| = 24 edges
Layer 3: M_3 = |V| = 4 nodes (0 added),|E| = 6 edges
Layer 4: M_4 = |V| = 2 n

PPI Shape : 46 46
Number of Edge : 607 // 70
Layer 0: M_0 = |V| = 48 nodes (2 added),|E| = 607 edges
Layer 1: M_1 = |V| = 24 nodes (1 added),|E| = 230 edges
Layer 2: M_2 = |V| = 12 nodes (0 added),|E| = 65 edges
Layer 3: M_3 = |V| = 6 nodes (0 added),|E| = 15 edges
Layer 4: M_4 = |V| = 3 nodes (0 added),|E| = 3 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 66 66
Number of Edge : 899 // 71
Layer 0: M_0 = |V| = 80 nodes (14 added),|E| = 899 edges
Layer 1: M_1 = |V| = 40 nodes (7 added),|E| = 367 edges
Layer 2: M_2 = |V| = 20 nodes (3 added),|E| = 126 edges
Layer 3: M_3 = |V| = 10 nodes (1 added),|E| = 36 edges
Layer 4: M_4 = |V| = 5 nodes (0 added),|E| = 10 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 69 69
Number of Edge : 1154 // 72
Layer 0: M_0 = |V| = 80 nodes (11 added),|E| = 1154 edges
Layer 1: M_1 = |V| = 40 nodes (5 added),|E| = 464 edges
Layer 2: M_2 = |V| = 20 nodes (2 added),|E| = 142 edges
Layer 3: M_3 = |V| = 10 nodes (1 added),|E| = 36 edges
Layer 4: M_

PPI Shape : 71 71
Number of Edge : 1173 // 93
Layer 0: M_0 = |V| = 80 nodes (9 added),|E| = 1173 edges
Layer 1: M_1 = |V| = 40 nodes (4 added),|E| = 460 edges
Layer 2: M_2 = |V| = 20 nodes (2 added),|E| = 139 edges
Layer 3: M_3 = |V| = 10 nodes (1 added),|E| = 36 edges
Layer 4: M_4 = |V| = 5 nodes (0 added),|E| = 10 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 89 89
Number of Edge : 2125 // 94
Layer 0: M_0 = |V| = 96 nodes (7 added),|E| = 2125 edges
Layer 1: M_1 = |V| = 48 nodes (3 added),|E| = 802 edges
Layer 2: M_2 = |V| = 24 nodes (1 added),|E| = 234 edges
Layer 3: M_3 = |V| = 12 nodes (0 added),|E| = 64 edges
Layer 4: M_4 = |V| = 6 nodes (0 added),|E| = 15 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 22 22
Number of Edge : 66 // 95
Layer 0: M_0 = |V| = 32 nodes (10 added),|E| = 66 edges
Layer 1: M_1 = |V| = 16 nodes (5 added),|E| = 29 edges
Layer 2: M_2 = |V| = 8 nodes (2 added),|E| = 11 edges
Layer 3: M_3 = |V| = 4 nodes (0 added),|E| = 5 edges
Layer 4: M_4 =

...

PPI Shape : 43 43
Number of Edge : 368 // 203
Layer 0: M_0 = |V| = 48 nodes (5 added),|E| = 368 edges
Layer 1: M_1 = |V| = 24 nodes (2 added),|E| = 135 edges
Layer 2: M_2 = |V| = 12 nodes (1 added),|E| = 45 edges
Layer 3: M_3 = |V| = 6 nodes (0 added),|E| = 14 edges
Layer 4: M_4 = |V| = 3 nodes (0 added),|E| = 3 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 109 109
Number of Edge : 2140 // 204
Layer 0: M_0 = |V| = 112 nodes (3 added),|E| = 2140 edges
Layer 1: M_1 = |V| = 56 nodes (1 added),|E| = 926 edges
Layer 2: M_2 = |V| = 28 nodes (0 added),|E| = 318 edges
Layer 3: M_3 = |V| = 14 nodes (0 added),|E| = 88 edges
Layer 4: M_4 = |V| = 7 nodes (0 added),|E| = 21 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 113 113
Number of Edge : 2396 // 205
Layer 0: M_0 = |V| = 128 nodes (15 added),|E| = 2396 edges
Layer 1: M_1 = |V| = 64 nodes (7 added),|E| = 1058 edges
Layer 2: M_2 = |V| = 32 nodes (3 added),|E| = 362 edges
Layer 3: M_3 = |V| = 16 nodes (1 added),|E| = 105 edges

../lib/coarsening.py:152: RuntimeWarning: divide by zero encountered in true_divide
  tval = vv[rs+jj] * (1.0/weights[tid] + 1.0/weights[nid])


PPI Shape : 66 66
Number of Edge : 1260 // 209
Layer 0: M_0 = |V| = 80 nodes (14 added),|E| = 1260 edges
Layer 1: M_1 = |V| = 40 nodes (7 added),|E| = 458 edges
Layer 2: M_2 = |V| = 20 nodes (3 added),|E| = 132 edges
Layer 3: M_3 = |V| = 10 nodes (1 added),|E| = 36 edges
Layer 4: M_4 = |V| = 5 nodes (0 added),|E| = 10 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 45 45
Number of Edge : 607 // 210
Layer 0: M_0 = |V| = 48 nodes (3 added),|E| = 607 edges
Layer 1: M_1 = |V| = 24 nodes (1 added),|E| = 230 edges
Layer 2: M_2 = |V| = 12 nodes (0 added),|E| = 65 edges
Layer 3: M_3 = |V| = 6 nodes (0 added),|E| = 15 edges
Layer 4: M_4 = |V| = 3 nodes (0 added),|E| = 3 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 58 58
Number of Edge : 921 // 211
Layer 0: M_0 = |V| = 64 nodes (6 added),|E| = 921 edges
Layer 1: M_1 = |V| = 32 nodes (2 added),|E| = 365 edges
Layer 2: M_2 = |V| = 16 nodes (1 added),|E| = 103 edges
Layer 3: M_3 = |V| = 8 nodes (0 added),|E| = 28 edges
Layer 4: M

PPI Shape : 86 86
Number of Edge : 1652 // 232
Layer 0: M_0 = |V| = 96 nodes (10 added),|E| = 1652 edges
Layer 1: M_1 = |V| = 48 nodes (4 added),|E| = 739 edges
Layer 2: M_2 = |V| = 24 nodes (2 added),|E| = 219 edges
Layer 3: M_3 = |V| = 12 nodes (1 added),|E| = 55 edges
Layer 4: M_4 = |V| = 6 nodes (0 added),|E| = 15 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 27 27
Number of Edge : 277 // 233
Layer 0: M_0 = |V| = 32 nodes (5 added),|E| = 277 edges
Layer 1: M_1 = |V| = 16 nodes (2 added),|E| = 90 edges
Layer 2: M_2 = |V| = 8 nodes (1 added),|E| = 21 edges
Layer 3: M_3 = |V| = 4 nodes (0 added),|E| = 6 edges
Layer 4: M_4 = |V| = 2 nodes (0 added),|E| = 1 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 119 119
Number of Edge : 4118 // 234
Layer 0: M_0 = |V| = 128 nodes (9 added),|E| = 4118 edges
Layer 1: M_1 = |V| = 64 nodes (4 added),|E| = 1549 edges
Layer 2: M_2 = |V| = 32 nodes (2 added),|E| = 430 edges
Layer 3: M_3 = |V| = 16 nodes (1 added),|E| = 105 edges
Layer

PPI Shape : 46 46
Number of Edge : 749 // 255
Layer 0: M_0 = |V| = 48 nodes (2 added),|E| = 749 edges
Layer 1: M_1 = |V| = 24 nodes (1 added),|E| = 240 edges
Layer 2: M_2 = |V| = 12 nodes (0 added),|E| = 66 edges
Layer 3: M_3 = |V| = 6 nodes (0 added),|E| = 15 edges
Layer 4: M_4 = |V| = 3 nodes (0 added),|E| = 3 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 21 21
Number of Edge : 184 // 256
Layer 0: M_0 = |V| = 32 nodes (11 added),|E| = 184 edges
Layer 1: M_1 = |V| = 16 nodes (5 added),|E| = 46 edges
Layer 2: M_2 = |V| = 8 nodes (2 added),|E| = 11 edges
Layer 3: M_3 = |V| = 4 nodes (1 added),|E| = 3 edges
Layer 4: M_4 = |V| = 2 nodes (0 added),|E| = 1 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 37 37
Number of Edge : 512 // 257
Layer 0: M_0 = |V| = 48 nodes (11 added),|E| = 512 edges
Layer 1: M_1 = |V| = 24 nodes (5 added),|E| = 166 edges
Layer 2: M_2 = |V| = 12 nodes (2 added),|E| = 45 edges
Layer 3: M_3 = |V| = 6 nodes (1 added),|E| = 10 edges
Layer 4: M_4 = |V|

In [59]:
data_IC50 = pd.read_csv("Data/Table_S6_GDSC_Drug_response_IC50.csv")
data_IC50.drop(['Unnamed: 0'], axis='columns', inplace=True)
print(data_IC50.shape)

(734, 201)


In [60]:
from scipy.sparse import coo_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats
n_fold = 3

Y_train, Y_test = train_test_split(data_IC50,  test_size=0.25, shuffle=True, random_state=20)
#print(Y_test.shape)


for cv in range(1):
    Y_pred = np.zeros([Y_test.shape[0], Y_test.shape[1]])
    Y_test = np.zeros([Y_test.shape[0], Y_test.shape[1]])
#    print('n_fold:{}'.format(cv))
    j = 0
    for i in range(1):
        label = data_IC50.iloc[:,i]
        label = np.array(label)
        data_minmax = label[~np.isnan(label)]
        min = data_minmax.min()
        max = data_minmax.max()
        label = (label - min) / (max - min)
#         label_indexes = np.argwhere([~np.isnan(label)])[:, 1]
        
        train_idx, test_idx = train_test_split(range(label.shape[0]), test_size=0.25, shuffle=True, random_state=20)
        train_idx = np.array(train_idx)
        
        train_idx = train_idx[~np.isnan(label[train_idx])]        
        list_train, list_val = Validation(n_fold,train_idx)

        val_idx = train_idx[list_val[cv]]
        train_idx = train_idx[list_train[cv]]
        test_idx = np.array(test_idx)
        
        train_labels = label[train_idx]
        val_labels = label[val_idx]
        test_labels = label[test_idx]
        
        
        common = {}
        common['num_epochs']     = 40
        common['batch_size']     = 4
        common['decay_steps']    = train_idx.shape[0] / common['batch_size']
        print(train_idx.shape[0], )
        common['eval_frequency'] = 10 * common['num_epochs']
        common['brelu']          = 'b1relu'
        common['pool']           = 'mpool1'

        common['regularization'] = 0
        common['dropout']        = 1
        common['learning_rate']  = 0.02
        common['decay_rate']     = 0.95
        common['momentum']       = 0.9
        Feature_size = []
        Kernel_size = []
        Pooling_size = []
        FC = []
        ConcatFC = []
        
        
        #######################################각 pathway별 파라미터 지정
        for i in range(len(dataset['L'])):   
            if dataset['exp'][i].shape[1] > 100: # 노드가 30개 이상인 
                Feature_size.append([40])
                Kernel_size.append([20])
                Pooling_size.append([2])
                FC.append([1])
                ConcatFC.append([1])
            elif dataset['exp'][i].shape[1] <= 100: # 노드가 5개 이상인..
                Feature_size.append([40])
                Kernel_size.append([10])
                Pooling_size.append([2])
                FC.append([1])
                ConcatFC.append([1])
        ##############################################
        common['F']              = Feature_size
        common['K']              = Kernel_size
        common['p']              = Pooling_size
        common['M']              = FC
        # concatnate fully connected NN
        common['M1']             = ConcatFC

        train_dataset = []
        val_dataset = []
        test_dataset = []
        for kegg_id in range(len(dataset['L'])):
            exp = dataset['exp'][kegg_id]
            perm = dataset['perm'][kegg_id]
#            print('kegg_id: {}, #nodes: {}'.format(kegg_id, len(perm)))
#            print(exp.shape)
#            print(train_idx.shape)
            train_data = exp[train_idx, :]
            val_data = exp[val_idx, :]
            test_data = exp[test_idx, :]
            train_data = coarsening.perm_data(train_data, perm)
            val_data = coarsening.perm_data(val_data, perm)
            test_data = coarsening.perm_data(test_data, perm)
            train_dataset.append(train_data)
            val_dataset.append(val_data)
            test_dataset.append(test_data)
#            print(train_data.shape, val_data.shape, test_data.shape)
        if True:
            name = 'cgconv_softmax'
            params = common.copy()
       
        model = models_Path.cgcnn(dataset['L'], **params)
        loss, t_step = model.fit(train_dataset, train_labels, val_dataset, val_labels)


        Y_pred[:, j] = model.predict(test_dataset)
        Y_test[:, j] = test_labels        
        print(Y_pred[:, j])
        print(Y_test[:, j])
        j = j+1
        np.savez(('PathwayTest.cv_{}'.format(cv)), Y_true=Y_test, Y_pred=Y_pred)

    np.savez(('PathwayTest.cv_{}'.format(cv)), Y_true=Y_test, Y_pred=Y_pred)


543
--------------------------------------!!
---0_th KEGG GCN ---
  input: M_0 = 48
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 48 * 40 / 2 = 960
    weights: F_0 * F_1 * K_1 = 1 * 40 * 10 = 400
    biases: F_1 = 40
  layer 2: logits (softmax)
    representation: M_2 = 1
    weights: M_1 * M_2 = 960 * 1 = 960
    biases: M_2 = 1
---1_th KEGG GCN ---
  input: M_0 = 32
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 32 * 40 / 2 = 640
    weights: F_0 * F_1 * K_1 = 1 * 40 * 10 = 400
    biases: F_1 = 40
  layer 2: logits (softmax)
    representation: M_2 = 1
    weights: M_1 * M_2 = 640 * 1 = 640
    biases: M_2 = 1
---2_th KEGG GCN ---
  input: M_0 = 32
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 32 * 40 / 2 = 640
    weights: F_0 * F_1 * K_1 = 1 * 40 * 10 = 400
    biases: F_1 = 40
  layer 2: logits (softmax)
    representation: M_2 = 1
    weights: M_1 * M_2 = 640 * 1 = 640
    biases: M_2 = 1
---3_th KEGG GCN ---
  input: M_0 = 32
  layer 1: cgconv1


---39_th KEGG GCN ---
  input: M_0 = 48
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 48 * 40 / 2 = 960
    weights: F_0 * F_1 * K_1 = 1 * 40 * 10 = 400
    biases: F_1 = 40
  layer 2: logits (softmax)
    representation: M_2 = 1
    weights: M_1 * M_2 = 960 * 1 = 960
    biases: M_2 = 1
---40_th KEGG GCN ---
  input: M_0 = 96
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 96 * 40 / 2 = 1920
    weights: F_0 * F_1 * K_1 = 1 * 40 * 10 = 400
    biases: F_1 = 40
  layer 2: logits (softmax)
    representation: M_2 = 1
    weights: M_1 * M_2 = 1920 * 1 = 1920
    biases: M_2 = 1
---41_th KEGG GCN ---
  input: M_0 = 64
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 64 * 40 / 2 = 1280
    weights: F_0 * F_1 * K_1 = 1 * 40 * 10 = 400
    biases: F_1 = 40
  layer 2: logits (softmax)
    representation: M_2 = 1
    weights: M_1 * M_2 = 1280 * 1 = 1280
    biases: M_2 = 1
---42_th KEGG GCN ---
  input: M_0 = 64
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1

step 400 / 5430 (epoch 2.95 / 40):
  learning_rate = 1.80e-02, loss_average = 4.07e-02
  validation loss: 4.37e-02
  time: 586s (wall 126s)
step 800 / 5430 (epoch 5.89 / 40):
  learning_rate = 1.55e-02, loss_average = 2.52e-02
  validation loss: 3.34e-02
  time: 1156s (wall 225s)
step 1200 / 5430 (epoch 8.84 / 40):
  learning_rate = 1.33e-02, loss_average = 3.18e-02
  validation loss: 3.41e-02
  time: 1688s (wall 288s)
step 1600 / 5430 (epoch 11.79 / 40):
  learning_rate = 1.14e-02, loss_average = 2.76e-02
  validation loss: 3.49e-02
  time: 2221s (wall 351s)
step 2000 / 5430 (epoch 14.73 / 40):
  learning_rate = 9.75e-03, loss_average = 2.77e-02
  validation loss: 3.32e-02
  time: 2752s (wall 414s)
step 2400 / 5430 (epoch 17.68 / 40):
  learning_rate = 8.36e-03, loss_average = 3.64e-02
  validation loss: 3.27e-02
  time: 3283s (wall 477s)
step 2800 / 5430 (epoch 20.63 / 40):
  learning_rate = 7.17e-03, loss_average = 3.26e-02
  validation loss: 3.36e-02
  time: 3813s (wall 539s)
step 

In [61]:
RMSE = []
PCC = []
SCC = []

for i in range(1):
    y_test = Y_test[:, i]
    y_pred = Y_pred[:, i]
    y_pred = y_pred[~np.isnan(y_test)]
    y_test = y_test[~np.isnan(y_test)]

    RMSE.append(mean_squared_error(y_test,y_pred))
    PCC.append(stats.pearsonr(y_test,y_pred)[0])
    SCC.append(stats.spearmanr(y_test,y_pred)[0])
print("RMSE :",RMSE)
print("PCC :",PCC)
print("SCC :",SCC)

RMSE : [0.03373800813579914]
PCC : [0.3220935959033511]
SCC : [0.2831856950317397]
